# Football World Championships Task (R)

This notebook uses `bracketeer` to model a FIFA World Cup 26-style task using teams actually in current 2026 qualification status.

Source (qualified teams): https://www.fifa.com/en/tournaments/mens/worldcup/canadamexicousa2026/articles/qualified-teams

- 42 currently qualified teams
- group stage (14 groups of 3)
- knockout stage from group winners

In [ ]:
if (!requireNamespace("pak", quietly = TRUE)) {
  install.packages("pak", repos = "https://cloud.r-project.org")
}

if (!requireNamespace("bracketeer", quietly = TRUE)) {
  pak::pak("bbtheo/bracketeer")
}

library(bracketeer)

In [ ]:
teams <- c(
  "Canada", "Mexico", "USA",
  "Australia", "IR Iran", "Japan", "Jordan", "Korea Republic", "Qatar", "Saudi Arabia", "Uzbekistan",
  "Algeria", "Cabo Verde", "Cote d'Ivoire", "Egypt", "Ghana", "Morocco", "Senegal", "South Africa", "Tunisia",
  "Curacao", "Haiti", "Panama",
  "Argentina", "Brazil", "Colombia", "Ecuador", "Paraguay", "Uruguay",
  "New Zealand",
  "Austria", "Belgium", "Croatia", "England", "France", "Germany", "Netherlands", "Norway", "Portugal", "Scotland", "Spain", "Switzerland"
)

world_championship_spec <- spec() |>
  round_robin("groups", groups = 14) |>
  single_elim("knockout", take = top_per_group(1))

preflight <- validate(world_championship_spec, n = length(teams))
stopifnot(isTRUE(preflight$ok))

trn <- build(world_championship_spec, teams)

set.seed(2026)
random_score <- function(max_goals = 5L, allow_draw = TRUE) {
  score <- sample.int(max_goals + 1L, size = 2L, replace = TRUE) - 1L
  if (!allow_draw && score[[1]] == score[[2]]) {
    winner_idx <- sample.int(2L, size = 1L)
    score[[winner_idx]] <- score[[winner_idx]] + 1L
  }
  as.numeric(score)
}

# Complete group-stage results, then materialize knockout
group_matches <- matches(trn, "groups", status = "all")
for (i in seq_len(nrow(group_matches))) {
  trn <- result(
    trn,
    "groups",
    match = group_matches$match_id[[i]],
    score = random_score(max_goals = 5L, allow_draw = TRUE)
  )
}
trn <- advance(trn, "groups")

status <- stage_status(trn)
knockout_row <- status[status$stage == "knockout", , drop = FALSE]
stopifnot(nrow(knockout_row) == 1L)
stopifnot(isTRUE(knockout_row$materialized[[1]]))
stopifnot(nrow(matches(trn, "knockout", status = "all")) == 15L)

# Play the knockout stage through to a champion
pending <- matches(trn, "knockout", status = "pending")
for (i in seq_len(nrow(pending))) {
  trn <- result(
    trn,
    "knockout",
    match = pending$match_id[[i]],
    score = random_score(max_goals = 5L, allow_draw = FALSE)
  )
}

champion <- winner(trn)
stopifnot(!is.na(champion))
cat("football world championships task passed:", champion, "\n")